In [1]:
import wikipedia
import wikipediaapi 
import datetime
import pandas as pd
import numpy as np
from sklearn import feature_extraction
from sklearn import model_selection
import nltk
from nltk import tokenize
from nltk import word_tokenize, pos_tag
import string
from keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import multiclass
from sklearn import metrics
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, GlobalMaxPool1D, Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
classes = ['sport', 'economy', 'politics', 'health', 'tourism', 
           'entertainment', 'culture', 'music', 'education', 
           'real estate', 'nutrition', 'science', 'history', 'technology'
          ]

In [3]:
def generate_data(classes, n):
    data = [];
    for i in classes:
#         topics = wikipedia.search(i, results = n)
#         for j in topics:
#         print(classes)
#         print("Class: ", i)
        for j in list(wikipedia.search(i,results = n)):
            wiki_wiki = wikipediaapi.Wikipedia('en')
            page_py = wiki_wiki.page(j)
            if(page_py.exists()):
#                 print("Class: ", i)
#                 print("Title: ", j)
#                     print("Section: ", page_py.sections[0].title)
#                     print("Text: ", page_py.text)
                for k in page_py.categories:
                    if i in k.lower(): 
#                         print("Class: ", i)
                        data.append((i, page_py.summary))
                        break
            else:
                continue
    return data

In [5]:
data = generate_data(classes, 10)

In [6]:
for i in classes:
    c = 0
    for j in range(0, len(data)):
        if(data[j][0] == i):
            c = c + 1
    print("%s %d" %(i, c))

sport 10
economy 4
politics 6
health 7
tourism 9
entertainment 5
culture 9
music 7
education 9
real estate 9
nutrition 8
science 8
history 6
technology 8


In [7]:
def create_data_csv_file(name, data):
    name = name + '.csv'
#         '_' + str(datetime.datetime.now()).replace(' ', '_').replace('.', '-').replace(':', '-') + 
#         '.csv'
    sep = '<>'
    cols = 'class' + sep + 'text\n'
    with open(name, 'w+', encoding='utf-8') as file:
        file.write(cols)
#         YELLOW,SMALL,STRETCH,ADULT,T
        for i in range(0, len(data)):
            line = data[i][0] + sep + data[i][1].replace('\n', ' ')

            file.write(line)
            file.write('\n')
    file.close()
            
#     print(text)

In [9]:
# create_data_csv_file('data_10', data)

In [10]:
df = pd.read_csv('data_10.csv', sep = '<>')
df

<ipython-input-10-79470950938f>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('data_10.csv', sep = '<>')


,class,text
0,sport,Sport pertains to any form of competitive phys...
1,sport,Sport was an American sports magazine. Launche...
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп..."
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...
4,sport,Sport diving is an underwater sport that uses ...
...,...,...
100,technology,"Fintech a portmanteau of ""financial technology..."
101,technology,TCL Technology (originally an abbreviation for...
102,technology,Space technology is technology for use in oute...
103,technology,A technology company (or tech company) is an e...


In [11]:
df['class'].value_counts()

sport            10
education         9
tourism           9
real estate       9
culture           9
technology        8
nutrition         8
science           8
music             7
health            7
politics          6
history           6
entertainment     5
economy           4
Name: class, dtype: int64

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Palja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
dict_class_index = {
    class_name: i for i, class_name in enumerate(df['class'].unique())
}

df['class num'] = df['class'].map(dict_class_index)

In [14]:
df

,class,text,class num
0,sport,Sport pertains to any form of competitive phys...,0
1,sport,Sport was an American sports magazine. Launche...,0
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп...",0
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...,0
4,sport,Sport diving is an underwater sport that uses ...,0
...,...,...,...
100,technology,"Fintech a portmanteau of ""financial technology...",13
101,technology,TCL Technology (originally an abbreviation for...,13
102,technology,Space technology is technology for use in oute...,13
103,technology,A technology company (or tech company) is an e...,13


In [15]:
dict_class_index

{'sport': 0,
 'economy': 1,
 'politics': 2,
 'health': 3,
 'tourism': 4,
 'entertainment': 5,
 'culture': 6,
 'music': 7,
 'education': 8,
 'real estate': 9,
 'nutrition': 10,
 'science': 11,
 'history': 12,
 'technology': 13}

In [20]:
import advertools as adv
stop_words = sorted(adv.stopwords['english'])
print(stop_words)

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'both', 'bottom', 'but', 'by', 'ca', 'call', 'can', 'cannot', 'could', 'did', 'do', 'does', 'doing', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'go', 'had', 'has', 'have', 'he', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon

In [21]:
def clear_text(text):
    new_text = text.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words])).str.lower()
    new_text = new_text.replace(r'[^\w\s]+', '', regex=True)
    return new_text

In [22]:
new_text = clear_text(df['text'])

In [24]:
df['clear text'] = new_text
df

,class,text,class num,clear text
0,sport,Sport pertains to any form of competitive phys...,0,sport pertains form competitive physical activ...
1,sport,Sport was an American sports magazine. Launche...,0,sport american sports magazine launched septem...
2,sport,"Sport, Sport, Sport (Russian: Спорт, спорт, сп...",0,sport sport sport russian спорт спорт спорт 19...
3,sport,Sport Lisboa e Benfica (Portuguese pronunciat...,0,sport lisboa e benfica portuguese pronunciatio...
4,sport,Sport diving is an underwater sport that uses ...,0,sport diving underwater sport uses recreationa...
...,...,...,...,...
100,technology,"Fintech a portmanteau of ""financial technology...",13,fintech portmanteau financial technology refer...
101,technology,TCL Technology (originally an abbreviation for...,13,tcl technology originally abbreviation telepho...
102,technology,Space technology is technology for use in oute...,13,space technology technology use outer space tr...
103,technology,A technology company (or tech company) is an e...,13,a technology company or tech company electroni...


In [25]:
newest_text = []
for i in range(0, len(df)):
    newest_text.append(word_tokenize(df['clear text'][i]))

In [26]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Palja\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [27]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Palja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
def my_lemmitezer(text):
    return pos_tag(word_tokenize(text))

In [29]:
tokenized_lemmitzed_text = []
for i in range(0, len(df)):
    tokenized_lemmitzed_text.append(my_lemmitezer(df['clear text'][i]))

In [30]:
df['tokenized lemmitzed text'] = tokenized_lemmitzed_text

In [31]:
stemmer = nltk.stem.PorterStemmer()

In [32]:
def my_stemmer(text):
    tokenized_stemmed_text = []
    tokens = word_tokenize(text)
    for token in tokens:
        tokenized_stemmed_text.append(stemmer.stem(token))
    return tokenized_stemmed_text

In [33]:
tokenized_stemmed_text = []
for i in range(0, len(df)):
    tokenized_stemmed_text.append(my_stemmer(df['clear text'][i]))

In [34]:
df['tokenized stemmed text'] = tokenized_stemmed_text
# df['tokenized stemmed text'] = df['tokenized stemmed text'].str.join(' ')

In [35]:
x = df['clear text']
x

0      sport pertains form competitive physical activ...
1      sport american sports magazine launched septem...
2      sport sport sport russian спорт спорт спорт 19...
3      sport lisboa e benfica portuguese pronunciatio...
4      sport diving underwater sport uses recreationa...
                             ...                        
100    fintech portmanteau financial technology refer...
101    tcl technology originally abbreviation telepho...
102    space technology technology use outer space tr...
103    a technology company or tech company electroni...
104    agricultural technology agrotechnology abbrevi...
Name: clear text, Length: 105, dtype: object

In [36]:
y = df['class num']
y

0       0
1       0
2       0
3       0
4       0
       ..
100    13
101    13
102    13
103    13
104    13
Name: class num, Length: 105, dtype: int64

In [37]:
x_tr, x_ft, y_tr, y_ft = model_selection.train_test_split(x, y, test_size=0.2, stratify=y)

In [38]:
x_tr

100    fintech portmanteau financial technology refer...
86     social science branches science devoted study ...
9      bbc sport sports division bbc providing nation...
11     the economy india middle income developing mar...
72     real estate appraisal property valuation land ...
                             ...                        
64     sex education known sexual education sexuality...
98     information technology it use computers create...
97     technologies artifacts systematic application ...
27     tourism travel pleasure business theory practi...
56     sony music entertainment sme known simply sony...
Name: clear text, Length: 84, dtype: object

In [39]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x_tr, y_tr, test_size=0.3, stratify=y_tr, random_state=5)

In [40]:
count_vectorizer_t = feature_extraction.text.TfidfVectorizer()

In [41]:
count_vectorizer_t.fit(x_train)

TfidfVectorizer()

In [42]:
x_train_t_tfidf = count_vectorizer_t.transform(x_train)
x_test_t_tfidf = count_vectorizer_t.transform(x_test)

In [43]:
model = svm.LinearSVC(max_iter=100000)

In [44]:
model_t = multiclass.OneVsRestClassifier(model)

In [45]:
model_t.fit(x_train_t_tfidf, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [46]:
y_test_predicted_t = model_t.predict(x_test_t_tfidf)

In [47]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_t), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,2,0,0,0,0,0,0,0,0,0,0,0,0,0
economy,0,1,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,2,0,0,0,0,0,0,0,0,0,0,0
health,0,0,0,2,0,0,0,0,0,0,0,0,0,0
tourism,0,0,0,0,2,0,0,0,0,0,0,0,0,0
entertainment,0,0,0,0,0,1,0,0,0,0,0,0,0,0
culture,0,0,0,0,0,0,1,1,0,0,0,0,0,0
music,0,0,0,0,0,0,0,2,0,0,0,0,0,0
education,0,0,0,0,0,0,0,0,2,0,0,0,0,0
real estate,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [48]:
print(metrics.classification_report(y_test, y_test_predicted_t))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       0.67      1.00      0.80         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       0.50      0.50      0.50         2
           7       0.67      1.00      0.80         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      0.50      0.67         2
          12       1.00      0.50      0.67         2
          13       0.67      1.00      0.80         2

    accuracy                           0.85        26
   macro avg       0.89      0.86      0.85        26
weighted avg       0.88      0.85      0.84        26



In [49]:
metrics.f1_score(y_test, y_test_predicted_t, average='macro')

0.8499999999999999

In [50]:
count_vectorizer_l = feature_extraction.text.TfidfVectorizer(tokenizer=my_lemmitezer)

In [51]:
count_vectorizer_l.fit(x_train)

C:\Users\Palja\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function my_lemmitezer at 0x0000020ED6BD84C0>)

In [52]:
x_train_l_tfidf = count_vectorizer_l.transform(x_train)
x_test_l_tfidf = count_vectorizer_l.transform(x_test)

In [53]:
model = svm.LinearSVC(max_iter=100000)

In [54]:
model_l = multiclass.OneVsRestClassifier(model)

In [55]:
model_l.fit(x_train_l_tfidf, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [56]:
y_test_predicted_l = model_l.predict(x_test_l_tfidf)

In [57]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_l), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,2,0,0,0,0,0,0,0,0,0,0,0,0,0
economy,0,1,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,2,0,0,0,0,0,0,0,0,0,0,0
health,0,0,0,2,0,0,0,0,0,0,0,0,0,0
tourism,0,0,0,0,2,0,0,0,0,0,0,0,0,0
entertainment,0,0,0,0,0,1,0,0,0,0,0,0,0,0
culture,0,0,0,0,0,0,1,1,0,0,0,0,0,0
music,0,0,0,0,0,0,0,2,0,0,0,0,0,0
education,0,0,0,0,0,0,0,0,2,0,0,0,0,0
real estate,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [58]:
print(metrics.classification_report(y_test, y_test_predicted_l))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       0.67      1.00      0.80         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       0.50      0.50      0.50         2
           7       0.67      1.00      0.80         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      0.50      0.67         2
          12       1.00      0.50      0.67         2
          13       0.67      1.00      0.80         2

    accuracy                           0.85        26
   macro avg       0.89      0.86      0.85        26
weighted avg       0.88      0.85      0.84        26



In [59]:
metrics.f1_score(y_test, y_test_predicted_l, average='macro')

0.8499999999999999

In [60]:
count_vectorizer_s = feature_extraction.text.TfidfVectorizer(tokenizer=my_stemmer)

In [61]:
count_vectorizer_s.fit(x_train)

C:\Users\Palja\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function my_stemmer at 0x0000020ED6BD83A0>)

In [62]:
x_train_s_tfidf = count_vectorizer_s.transform(x_train)
x_test_s_tfidf = count_vectorizer_s.transform(x_test)

In [63]:
model = svm.LinearSVC(max_iter=100000)

In [64]:
model_s = multiclass.OneVsRestClassifier(model)

In [65]:
model_s.fit(x_train_s_tfidf, y_train)

OneVsRestClassifier(estimator=LinearSVC(max_iter=100000))

In [66]:
y_test_predicted_s = model_s.predict(x_test_s_tfidf)

In [67]:
cm = pd.DataFrame(metrics.confusion_matrix(y_test, y_test_predicted_s), columns=df['class'].unique(), index=df['class'].unique())
cm

,sport,economy,politics,health,tourism,entertainment,culture,music,education,real estate,nutrition,science,history,technology
sport,2,0,0,0,0,0,0,0,0,0,0,0,0,0
economy,0,1,0,0,0,0,0,0,0,0,0,0,0,0
politics,0,0,2,0,0,0,0,0,0,0,0,0,0,0
health,0,0,0,2,0,0,0,0,0,0,0,0,0,0
tourism,0,0,0,0,2,0,0,0,0,0,0,0,0,0
entertainment,0,0,0,0,0,1,0,0,0,0,0,0,0,0
culture,0,0,0,0,0,0,1,1,0,0,0,0,0,0
music,0,0,0,0,0,0,0,2,0,0,0,0,0,0
education,0,0,0,0,0,0,0,0,2,0,0,0,0,0
real estate,0,0,0,0,0,0,0,0,0,2,0,0,0,0


In [68]:
print(metrics.classification_report(y_test, y_test_predicted_s))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         2
           3       0.67      1.00      0.80         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       0.33      0.50      0.40         2
           7       0.67      1.00      0.80         2
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         2
          10       1.00      0.50      0.67         2
          11       1.00      0.50      0.67         2
          12       0.00      0.00      0.00         2
          13       0.67      1.00      0.80         2

    accuracy                           0.81        26
   macro avg       0.81      0.82      0.80        26
weighted avg       0.79      0.81      0.78        26



C:\Users\Palja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Palja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Palja\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
metrics.f1_score(y_test, y_test_predicted_s, average='macro')

0.7952380952380952